In [ ]:
import requests
from bs4 import BeautifulSoup

urls = {
    'Towards Data Science': 'https://towardsdatascience.com/archive/{0}/{1:02d}/{2:02d}',
}

In [ ]:
# takes the number day of the year and returns month, day
def convert_day(day):
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [ ]:
# converts the claps string
def get_claps(claps_str):
    if (claps_str is None) or (claps_str == '') or (claps_str.split is None):
        return 0
    split = claps_str.split('K')
    claps = float(split[0])
    claps = int(claps*1000) if len(split) == 2 else int(claps)
    return claps

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

/Users/yujiantang/Documents/workspace/chat_articles/chatter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import bs4
import time
# data_batch = []
year = 2023
for i in range(201, 243):
    month, day = convert_day(i)
    date = '{0}-{1:02d}-{2:02d}'.format(year, month, day)
    for publication, url in urls.items():
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        time.sleep(8)
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all(
            "div",
            class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        for article in articles:
            title = article.find("h3", class_="graf--title")
            # print(title.contents)
            if title is None:
                continue
            title = str(title.contents[0]).replace(u'\xA0', u' ').replace(u'\u200a', u' ')
            # title = title.contents[0]
            print(title)
            subtitle = article.find("h4", class_="graf--subtitle")
            # print(subtitle.contents if subtitle is not None else " ")
            subtitle = str(subtitle.contents[0]).replace(u'\xA0', u' ').replace(u'\u200a', u' ') if subtitle is not None else ''
            article_url = article.find_all("a")[3]['href'].split('?')[0]
            try:
                claps = get_claps(article.find_all("button")[1].contents[0])
            except:
                claps = 0
            # print(claps)
            reading_time = article.find("span", class_="readingTime")
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
            responses = article.find_all("a")
            if len(responses) == 7:
                responses = responses[6].contents[0].split(' ')
                if len(responses) == 0:
                    responses = 0
                else:
                    responses = responses[0]
            else:
                responses = 0
            article_res = requests.get(article_url)
            time.sleep(8)

            if not article_res.ok:
                print(article_res)
                continue
            soup = bs4.BeautifulSoup(article_res.text)
            paragraphs = soup.select('[class*="pw-post-body-paragraph"]')
            for i, paragraph in enumerate(paragraphs):
                embedding = model.encode(paragraph.text)
                data_batch.append({
                    "_id": f"{article_url}+{i}",
                    "article_url": article_url,
                    "title": title,
                    "subtitle": subtitle,
                    "claps": claps,
                    "responses": responses,
                    "reading_time": reading_time,
                    "publication": publication,
                    "date": date,
                    "paragraph": paragraph.text,
                    "embedding": embedding
                })

How to Chunk Text Data — A Comparative Analysis
Understanding Large Language Models: The Physics of (Chat)GPT and BERT
ReLoRa: Pre-train a Large Language Model on Your GPU
A Data Science Project with ChatGPT Code Interpreter
Microsoft’s New AI Method to Predict How Molecules Move and Function
10 Ways to Add a Column to Pandas DataFrames
How to Fix Missing Dates for Time Series Analysis
Understand your Data in Real-Time
Map-Matching for Trajectory Prediction
Gradient Boosting: a Silver Bullet in Forecasting
Efficient k-Nearest Neighbors (k-NN) Solutions with NumPy
How to Look at Common Machine Learning Tasks with Fresh Eyes
Deep Dive into PFI for Model Interpretability
Does Your LLM Pipeline Achieve Your Goal?
Increasing the Reliability of Wireless Sensor Networks by Blind Calibration
How I built a programming language: The (difficult) Path to Success
Different ways of training LLMs
How to Build a 5-Layer Data Stack
How You Should Validate Machine Learning Models
4 Easy Ways to Instantl

In [ ]:
uniques = set()
for data in data_batch:
    if data['article_url'] not in uniques:
        uniques.add(data['article_url'])
len(uniques)

1384

In [ ]:
import pickle

In [ ]:
filename="TDS_8_30_2023"
# with open(f'{filename}.pkl', 'wb') as f:
#     pickle.dump(data_batch, f)
        
with open(f'{filename}.pkl', 'rb') as f:
    data_batch = pickle.load(f)

In [ ]:
data_batch[-1]

{'_id': 'https://towardsdatascience.com/how-to-one-way-anova-by-hand-4c19e2a61a8c+6',
 'article_url': 'https://towardsdatascience.com/how-to-one-way-anova-by-hand-4c19e2a61a8c',
 'title': 'How To: One-Way ANOVA by Hand',
 'subtitle': 'Learn how to perform a one-way Analysis Of VAriance (ANOVA) by hand to…',
 'claps': 30,
 'responses': 0,
 'reading_time': 6,
 'publication': 'Towards Data Science',
 'date': '2023-08-30',
 'paragraph': 'Note that the ANOVA requires some assumptions (i.e., independence, equality of variances and normality). The aim of this post is to illustrate how to do an ANOVA by hand and not how to verify these assumptions, so we suppose they are met without any verification. See how to test these assumptions in R if you are interested.',
 'embedding': array([ 1.02438033e-02,  2.63246838e-02,  3.95597294e-02, -3.51585522e-02,
        -6.38489425e-02,  3.91995534e-03, -6.00639731e-02,  1.37220668e-02,
        -6.36992082e-02, -1.08909970e-02,  3.19623463e-02, -2.5974167

In [ ]:
# "_id": f"{article_url}+{i}",
# "article_url": article_url,
# "title": title,
# "subtitle": subtitle,
# "claps": claps,
# "responses": responses,
# "reading_time": reading_time,
# "publication": publication,
# "date": date,
# "paragraph": paragraph.text,
# "embedding": embedding
for index, data in enumerate(data_batch):
    if type(data["responses"]) is not int:
        print(f"response error at index {index}")